<center><img src="images/header1.png" width=400></center>
<h1><center>Основы машинного обучения</center></h1>
<hr>
<h2><center>Анализ графов и социальных сетей</center></h2>
<h3><center>Ефимов Владислав</center></h3>

In [ ]:
!pip install vk

In [ ]:
import vk
import json

In [ ]:
data_path = "./data"

Сначала получаем токен:
* Нужно [создать](https://vk.com/apps?act=manage) свое тестовое приложение.
* Включить, сделать публичным (в настройках).
* Авторизоваться: из настроек скопировать client_id (ID приложения), вставить в параметр client_id вместо 0 и перейти по получившейся ссылке в браузере. В данном случае мы используем scope=friends, его нам будет достаточно, однако можно получить доступ ко всем необходимым частям аккаунта, список скоупов можно найти [тут](https://dev.vk.com/reference/access-rights).
> ```https://oauth.vk.com/authorize?client_id=0&display=page&redirect_uri=http://oauth.vk.com/blank.html&scope=friends&response_type=token```
* Скопировать значение поля access_token из обновленной адресной строки (если вы авторизованы уже были в браузере в вк, то вас сразу средиректит, иначе надо будет сначала залогиниться).
* <b>!!! Не светите нигде полученный токен, по нему можно получить полный доступ к аккаунту!!!</b>
* Важный момент: токен не заработает в колабе и скорее всего в ноутбуке на внешнем сервере, так как он привязывается к ip адресу, где он был выдан.

In [ ]:
token = ""

In [ ]:
api = vk.API(access_token = token)

In [ ]:
# Ваш id ВКонтакте, можно посмотреть в настройках
VKID = ...

Все готово! Доступные методы и их параметры можно найти в [официальной доке](https://dev.vk.com/reference).

Получим свой список друзей для начала.

In [ ]:
# Все доступные поля для v=5.131 версии апи, выкинул только различные ссылки на аватарку
fields = "bdate,can_post,can_see_all_posts,can_write_private_message,city,contacts,country,domain,education,has_mobile,timezone,last_seen,nickname,online,relation,sex,status,universities"

In [ ]:
friends = api.friends.get(user_id=VKID, fields=fields, v=5.131)

In [ ]:
friends

В ответе count -- число друзей (item-ов), items -- собственно сам список друзей с инфой о них. Сохраним json с данными.

In [ ]:
with open(f"{data_path}/friends_{VKID}.json", "w") as f:
    json.dump(friends["items"], f, indent=2)

Теперь получим общих друзей. Единственная тонкость в текущей версии апи: если страница друга удалена, то апи вернет ошибку на весь запрос. Поэтому исключим таких пользователей (у них появляется поле deactivated в инфо).

In [ ]:
common_friends = api.friends.getMutual(
    source_uid=VKID, target_uids=[fr["id"] for fr in friends["items"] if "deactivated" not in fr], v=5.131)

In [ ]:
# Переделаем немного структуру из ответа и сохраним
with open(f"{data_path}/common_friends_{VKID}.json", "w") as f:
    json.dump({x["id"]: x["common_friends"] for x in common_friends}, f, indent=2)